# Inference of the mit-b4 SegFormer

This is a notebook used for performing the actual inference in the HuBMAP-HPA public and private test sets of the kaggle competition. It used different mask thresholds per organ category and its final Dice scores were:
- Public Score: <span style="font-size: 130%;"><b>0.57659</b></span>
- Private Score: <span style="font-size: 130%;"><b>0.50252</b></span>

Some technical details:

&minus; The competition required that there could be no internet connection, so the following dataset was used for installing the required libraries:
[OpenMMLab Essential Repositories](https://www.kaggle.com/datasets/maxvandijck/openmmlab-essential-repositories)

&minus; Additionally, the dataset with the produced mit-b4 SegFormer model was added: [Model SegFormer mitb4 93.22 mDice](https://www.kaggle.com/datasets/theo88/segformer-mitb4-93-mdice).

&minus; In order to define different thresholds per organ the `MMSegmentation` original encoder-decoder code had to be tweeked. The alterations were taken from the [customize_mmseg](https://www.kaggle.com/code/opusen/customize-mmseg) notebook by [opusen](https://www.kaggle.com/opusen) and adapted to the `MMSegmentation` version used in this notebook.



## Installation of all the required libraries

<a id="section-one"></a>
### 1. PyTorch Installation

In [1]:
# sys prerequisite import
import sys
# pytorch version compatible with openmmlab
!pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/torch-1.10.0+cu111-cp37-cp37m-linux_x86_64.whl

Processing /kaggle/input/openmmlab-essential-repositories/openmmlab-repos/src/torch-1.10.0+cu111-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.9.1
    Uninstalling torch-1.9.1:
      Successfully uninstalled torch-1.9.1


<a id="section-two"></a>
### 2. MMCV Installation

In [2]:
# mmcv install
!pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/opencv_python-4.6.0.66-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

!pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/yapf-0.32.0-py2.py3-none-any.whl
!pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/addict-2.4.0-py3-none-any.whl
!pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/mmcv_full-1.5.3-cp37-cp37m-manylinux1_x86_64.whl

Processing /kaggle/input/openmmlab-essential-repositories/openmmlab-repos/src/opencv_python-4.6.0.66-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.4.60
    Uninstalling opencv-python-4.5.4.60:
      Successfully uninstalled opencv-python-4.5.4.60
Processing /kaggle/input/openmmlab-essential-repositories/openmmlab-repos/src/yapf-0.32.0-py2.py3-none-any.whl
Processing /kaggle/input/openmmlab-essential-repositories/openmmlab-repos/src/addict-2.4.0-py3-none-any.whl
Processing /kaggle/input/openmmlab-essential-repositories/openmmlab-repos/src/mmcv_full-1.5.3-cp37-cp37m-manylinux1_x86_64.whl


<a id="section-three"></a>
### 3. MMClassification Installation
* **Requires MMCV**

In [3]:
# mmclassification install
!cp -r ../input/openmmlab-essential-repositories/openmmlab-repos/mmclassification /kaggle/working
%cd /kaggle/working/mmclassification
!pip install -e .
%cd ..

sys.path.append('./mmclassification')

/kaggle/working/mmclassification
Obtaining file:///kaggle/working/mmclassification
  Preparing metadata (setup.py) ... - \ done
  Running setup.py develop for mmcls
/kaggle/working


<a id="section-four"></a>
### 4. MMSegmentation Installation
* **Requires MMCV**
* **Requires MMClassification**

In [5]:
## mmsegmentation install
!cp -r ../input/openmmlab-essential-repositories/openmmlab-repos/mmsegmentation /kaggle/working/
%cd /kaggle/working/mmsegmentation
!pip install -e .
%cd ..

/kaggle/working/mmsegmentation
Obtaining file:///kaggle/working/mmsegmentation
  Preparing metadata (setup.py) ... - \ done
  Running setup.py develop for mmsegmentation
/kaggle/working


## Tweaking the original encoder-decoder
This teak was necessary in order for the model to return logits, so we could use them for different mask thresholds per organ category.

In [7]:
%%writefile /kaggle/working/mmsegmentation/mmseg/models/segmentors/encoder_decoder.py 
# Copyright (c) OpenMMLab. All rights reserved.
import torch
import torch.nn as nn
import torch.nn.functional as F

from mmseg.core import add_prefix
from mmseg.ops import resize
from .. import builder
from ..builder import SEGMENTORS
from .base import BaseSegmentor


@SEGMENTORS.register_module()
class EncoderDecoder(BaseSegmentor):
    """Encoder Decoder segmentors.

    EncoderDecoder typically consists of backbone, decode_head, auxiliary_head.
    Note that auxiliary_head is only used for deep supervision during training,
    which could be dumped during inference.
    """

    def __init__(self,
                 backbone,
                 decode_head,
                 neck=None,
                 auxiliary_head=None,
                 train_cfg=None,
                 test_cfg=None,
                 pretrained=None,
                 init_cfg=None):
        super(EncoderDecoder, self).__init__(init_cfg)
        if pretrained is not None:
            assert backbone.get('pretrained') is None, \
                'both backbone and segmentor set pretrained weight'
            backbone.pretrained = pretrained
        self.backbone = builder.build_backbone(backbone)
        if neck is not None:
            self.neck = builder.build_neck(neck)
        self._init_decode_head(decode_head)
        self._init_auxiliary_head(auxiliary_head)

        self.train_cfg = train_cfg
        self.test_cfg = test_cfg

        assert self.with_decode_head

    def _init_decode_head(self, decode_head):
        """Initialize ``decode_head``"""
        self.decode_head = builder.build_head(decode_head)
        self.align_corners = self.decode_head.align_corners
        self.num_classes = self.decode_head.num_classes

    def _init_auxiliary_head(self, auxiliary_head):
        """Initialize ``auxiliary_head``"""
        if auxiliary_head is not None:
            if isinstance(auxiliary_head, list):
                self.auxiliary_head = nn.ModuleList()
                for head_cfg in auxiliary_head:
                    self.auxiliary_head.append(builder.build_head(head_cfg))
            else:
                self.auxiliary_head = builder.build_head(auxiliary_head)

    def extract_feat(self, img):
        """Extract features from images."""
        x = self.backbone(img)
        if self.with_neck:
            x = self.neck(x)
        return x

    def encode_decode(self, img, img_metas):
        """Encode images with backbone and decode into a semantic segmentation
        map of the same size as input."""
        x = self.extract_feat(img)
        out = self._decode_head_forward_test(x, img_metas)
        out = resize(
            input=out,
            size=img.shape[2:],
            mode='bilinear',
            align_corners=self.align_corners)
        return out

    def _decode_head_forward_train(self, x, img_metas, gt_semantic_seg):
        """Run forward function and calculate loss for decode head in
        training."""
        losses = dict()
        loss_decode = self.decode_head.forward_train(x, img_metas,
                                                     gt_semantic_seg,
                                                     self.train_cfg)

        losses.update(add_prefix(loss_decode, 'decode'))
        return losses

    def _decode_head_forward_test(self, x, img_metas):
        """Run forward function and calculate loss for decode head in
        inference."""
        seg_logits = self.decode_head.forward_test(x, img_metas, self.test_cfg)
        return seg_logits

    def _auxiliary_head_forward_train(self, x, img_metas, gt_semantic_seg):
        """Run forward function and calculate loss for auxiliary head in
        training."""
        losses = dict()
        if isinstance(self.auxiliary_head, nn.ModuleList):
            for idx, aux_head in enumerate(self.auxiliary_head):
                loss_aux = aux_head.forward_train(x, img_metas,
                                                  gt_semantic_seg,
                                                  self.train_cfg)
                losses.update(add_prefix(loss_aux, f'aux_{idx}'))
        else:
            loss_aux = self.auxiliary_head.forward_train(
                x, img_metas, gt_semantic_seg, self.train_cfg)
            losses.update(add_prefix(loss_aux, 'aux'))

        return losses

    def forward_dummy(self, img):
        """Dummy forward function."""
        seg_logit = self.encode_decode(img, None)

        return seg_logit

    def forward_train(self, img, img_metas, gt_semantic_seg):
        """Forward function for training.

        Args:
            img (Tensor): Input images.
            img_metas (list[dict]): List of image info dict where each dict
                has: 'img_shape', 'scale_factor', 'flip', and may also contain
                'filename', 'ori_shape', 'pad_shape', and 'img_norm_cfg'.
                For details on the values of these keys see
                `mmseg/datasets/pipelines/formatting.py:Collect`.
            gt_semantic_seg (Tensor): Semantic segmentation masks
                used if the architecture supports semantic segmentation task.

        Returns:
            dict[str, Tensor]: a dictionary of loss components
        """

        x = self.extract_feat(img)

        losses = dict()

        loss_decode = self._decode_head_forward_train(x, img_metas,
                                                      gt_semantic_seg)
        losses.update(loss_decode)

        if self.with_auxiliary_head:
            loss_aux = self._auxiliary_head_forward_train(
                x, img_metas, gt_semantic_seg)
            losses.update(loss_aux)

        return losses

    # TODO refactor
    def slide_inference(self, img, img_meta, rescale):
        """Inference by sliding-window with overlap.

        If h_crop > h_img or w_crop > w_img, the small patch will be used to
        decode without padding.
        """

        h_stride, w_stride = self.test_cfg.stride
        h_crop, w_crop = self.test_cfg.crop_size
        batch_size, _, h_img, w_img = img.size()
        num_classes = self.num_classes
        h_grids = max(h_img - h_crop + h_stride - 1, 0) // h_stride + 1
        w_grids = max(w_img - w_crop + w_stride - 1, 0) // w_stride + 1
        preds = img.new_zeros((batch_size, num_classes, h_img, w_img))
        count_mat = img.new_zeros((batch_size, 1, h_img, w_img))
        for h_idx in range(h_grids):
            for w_idx in range(w_grids):
                y1 = h_idx * h_stride
                x1 = w_idx * w_stride
                y2 = min(y1 + h_crop, h_img)
                x2 = min(x1 + w_crop, w_img)
                y1 = max(y2 - h_crop, 0)
                x1 = max(x2 - w_crop, 0)
                crop_img = img[:, :, y1:y2, x1:x2]
                crop_seg_logit = self.encode_decode(crop_img, img_meta)
                preds += F.pad(crop_seg_logit,
                               (int(x1), int(preds.shape[3] - x2), int(y1),
                                int(preds.shape[2] - y2)))

                count_mat[:, :, y1:y2, x1:x2] += 1
        assert (count_mat == 0).sum() == 0
        if torch.onnx.is_in_onnx_export():
            # cast count_mat to constant while exporting to ONNX
            count_mat = torch.from_numpy(
                count_mat.cpu().detach().numpy()).to(device=img.device)
        preds = preds / count_mat
        if rescale:
            # remove padding area
#             resize_shape = img_meta[0]['img_shape'][:2]
#             preds = preds[:, :, :resize_shape[0], :resize_shape[1]]
            preds = resize(
                preds,
                size=img_meta[0]['ori_shape'][:2],
                mode='bilinear',
                align_corners=self.align_corners,
                warning=False)
        return preds

    def whole_inference(self, img, img_meta, rescale):
        """Inference with full image."""

        seg_logit = self.encode_decode(img, img_meta)
        if rescale:
            # support dynamic shape for onnx
            if torch.onnx.is_in_onnx_export():
                size = img.shape[2:]
            else:
                # remove padding area
#                 resize_shape = img_meta[0]['img_shape'][:2]
#                 seg_logit = seg_logit[:, :, :resize_shape[0], :resize_shape[1]]
                size = img_meta[0]['ori_shape'][:2]
            seg_logit = resize(
                seg_logit,
                size=size,
                mode='bilinear',
                align_corners=self.align_corners,
                warning=False)

        return seg_logit

    def inference(self, img, img_meta, rescale):
        """Inference with slide/whole style.

        Args:
            img (Tensor): The input image of shape (N, 3, H, W).
            img_meta (dict): Image info dict where each dict has: 'img_shape',
                'scale_factor', 'flip', and may also contain
                'filename', 'ori_shape', 'pad_shape', and 'img_norm_cfg'.
                For details on the values of these keys see
                `mmseg/datasets/pipelines/formatting.py:Collect`.
            rescale (bool): Whether rescale back to original shape.

        Returns:
            Tensor: The output segmentation map.
        """

        assert self.test_cfg.mode in ['slide', 'whole']
        ori_shape = img_meta[0]['ori_shape']
        assert all(_['ori_shape'] == ori_shape for _ in img_meta)
        if self.test_cfg.mode == 'slide':
            seg_logit = self.slide_inference(img, img_meta, rescale)
        else:
            seg_logit = self.whole_inference(img, img_meta, rescale)

        if self.test_cfg.get('sigmoid', False):
            output = F.sigmoid(seg_logit)
        else:
            output = F.softmax(seg_logit, dim=1)
        flip = img_meta[0]['flip']
        if flip:
            flip_direction = img_meta[0]['flip_direction']
            assert flip_direction in ['horizontal', 'vertical']
            if flip_direction == 'horizontal':
                output = output.flip(dims=(3, ))
            elif flip_direction == 'vertical':
                output = output.flip(dims=(2, ))

        return output

    def simple_test(self, img, img_meta, rescale=True):
        """Simple test with single image."""
        seg_logit = self.inference(img, img_meta, rescale)
        if self.test_cfg.get('sigmoid', False):
            seg_pred = seg_logit
        else:
            seg_pred = seg_logit.argmax(dim=1)
        if torch.onnx.is_in_onnx_export():
            # our inference backend only support 4D output
            seg_pred = seg_pred.unsqueeze(0)
            return seg_pred
        seg_pred = seg_pred.cpu().numpy()
        # unravel batch dim
        seg_pred = list(seg_pred)
        return seg_pred

    def aug_test(self, imgs, img_metas, rescale=True):
        """Test with augmentations.

        Only rescale=True is supported.
        """
        # aug_test rescale all imgs back to ori_shape for now
        assert rescale
        # to save memory, we get augmented seg logit inplace
        seg_logit = self.inference(imgs[0], img_metas[0], rescale)
        for i in range(1, len(imgs)):
            cur_seg_logit = self.inference(imgs[i], img_metas[i], rescale)
            seg_logit += cur_seg_logit
        seg_logit /= len(imgs)
        if self.test_cfg.get('sigmoid', False):
            seg_pred = seg_logit
        else:
            seg_pred = seg_logit.argmax(dim=1)
        seg_pred = seg_pred.cpu().numpy()
        # unravel batch dim
        seg_pred = list(seg_pred)
        return seg_pred

Overwriting /kaggle/working/mmsegmentation/mmseg/models/segmentors/encoder_decoder.py


## Performing Inference with different thresholds

In [8]:
import cv2
import numpy as np
import pandas as pd
import os
from glob import glob
from tqdm.notebook import tqdm
import gc
sys.path.append('./mmsegmentation')

from mmseg.apis import init_segmentor, inference_segmentor
from mmcv.utils import config

In [9]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

We have to change the original configuration file in order for the tweak to be used by setting `test_cfg=dict(mode='whole', sigmoid=True)`

In [ ]:
config_file = '/kaggle/input/segformer-mitb4-93-mdice/segformer_HuBMAP.py'
new_config = '/kaggle/working/segformer_HuBMAP.py'
!cp {config_file} {new_config}
!sed -i "s/test_cfg=dict(mode='whole'))/test_cfg=dict(mode='whole', sigmoid=True))/" {new_config}
# Correcting a typo in the configuration file of the model dataset
!sed -i "s/num_classes=6/num_classes=2/" {new_config}
!grep test_cfg {new_config}
!grep num_classes {new_config}

In [11]:
DATA = '/kaggle/input/hubmap-organ-segmentation/test_images/'
df_sample = pd.read_csv('/kaggle/input/hubmap-organ-segmentation/sample_submission.csv').set_index('id')
df_test = pd.read_csv('/kaggle/input/hubmap-organ-segmentation/test.csv')
checkpoint_file = '/kaggle/input/segformer-mitb4-93-mdice/SegFormer_mitb4_1024x1024_9322_No_Albu.pth'

# build the model from a config file and a checkpoint file
model = init_segmentor(new_config, checkpoint_file, device='cuda:0')

./mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '


load checkpoint from local path: /kaggle/input/segformer-mitb4-93-mdice/SegFormer_mitb4_1024x1024_9322_No_Albu.pth


In [12]:
thresholds={
    "kidney":0.225,
    "prostate":0.225,
    "largeintestine":0.225,
    "spleen":0.1,
    "lung":0.1}

In [13]:
names,preds = [],[]
imgs, pd_mks = [],[]
debug = len(df_sample)<2
for idx,row in tqdm(df_sample.iterrows(),total=len(df_sample)):
    img_file = os.path.join(DATA,str(idx)+'.tiff')
    img_inf = inference_segmentor(model, img_file)
    pred = img_inf[0]
    organ = df_test.loc[df_test.id==idx, 'organ'].item()
    thresh = thresholds[organ]
    pred = (pred[1]>=thresh).astype(np.uint8)
    rle = rle_encode_less_memory(pred)
    names.append(str(idx))
    preds.append(rle)
    if debug:
        img  = cv2.imread(os.path.join(DATA,str(idx)+'.tiff'))
        imgs.append(img)
        pd_mks.append(pred)
    del img_inf, pred, rle, idx, row
    gc.collect()

  0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [14]:
if debug:
    import matplotlib.pyplot as plt
    from mmseg.apis import show_result_pyplot
    for img, mask in zip(imgs, pd_mks):
        plt.figure(figsize=(12, 7))
        plt.subplot(1, 3, 1); plt.imshow(img); plt.axis('OFF'); plt.title('image')
        plt.subplot(1, 3, 2); plt.imshow(mask*255); plt.axis('OFF'); plt.title('mask')
        plt.subplot(1, 3, 3); plt.imshow(img); plt.imshow(mask*255, alpha=0.4); plt.axis('OFF'); plt.title('overlay')
        plt.tight_layout()
        plt.show()

In [15]:
if not debug:
    !rm -rf /kaggle/working/*

In [16]:
df = pd.DataFrame({'id':names,'rle':preds})
df.to_csv('submission.csv',index=False)